In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Cross-Validation and Bias-Variance decomposition
## Cross-Validation
Implementing 4-fold cross-validation below:

In [3]:
from helpers import load_data

# load dataset
x, y = load_data()

In [1]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)



In [4]:
from costs import compute_mse
from ridge_regression import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    x_test=x[k_indices[k-1]]
    y_test=y[k_indices[k-1]]
    
    x_tr=np.delete(x,[k_indices[k-1]])
    y_tr=np.delete(y,[k_indices[k-1]])
    # ***************************************************
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form data with polynomial degree: TODO
    
    x_trpoly=build_poly(x_tr,degree)
    x_testpoly=build_poly(x_test,degree)
    # ***************************************************
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    wr_tr=ridge_regression(y_tr,x_trpoly,lambda_)
    
    mse_test=compute_mse(y_test,x_testpoly,wr_tr)         # loss pour les data test avec w training
    mse_tr=compute_mse(y_tr,x_trpoly,wr_tr)
    # ***************************************************
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate the loss for train and test data: TODO
    loss_te=np.power(mse_test*2,1/2)
    loss_tr=np.power(mse_tr*2,1/2)
    # ***************************************************
   
    return loss_tr, loss_te


In [ ]:
from plots import cross_validation_visualization

def cross_validation_demo():
    seed =2
    degree = 7
    k_fold = 4 
    lambdas = np.logspace(-4, 4, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    for lamb in lambdas:
        avtr=0
        avte=0
        for k in range(k_fold):
            rmsetr,rmsete=cross_validation(y,x,k_indices,k,lamb, degree)
            avtr=avtr+rmsetr
            avte=avte+rmsete
        rmse_tr.append(avtr/(k_fold))
        rmse_te.append(avte/(k_fold))
    # ***************************************************    
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)

cross_validation_demo()

In [6]:
def bestmodeldegree(y,x,lambda_,degree):
    degree=[i for i in range(degree)]
    k_fold = 4 
    seed =2
    k_indices = build_k_indices(y, k_fold, seed)

    rmse_tr = []
    rmse_te = []
    for deg in degree:
        avtr=0
        avte=0
        for k in range(k_fold):
            rmsetr,rmsete=cross_validation(y,x,k_indices,k,lambda_, deg)
            avtr=avtr+rmsetr
            avte=avte+rmsete
        rmse_tr.append(avtr/(k_fold))
        rmse_te.append(avte/(k_fold))
        
    plt.figure(figsize=(10,6))
    plt.plot(degree,rmse_tr,'*-',label='Train error')
    plt.plot(degree,rmse_te,'*-',label='Test error')
    #plt.xscale('log')
    plt.xlabel('degree')
    plt.grid()
    plt.legend(loc='best')

bestmodeldegree(y,x,10**12,11)

## Bias-Variance Decomposition
Visualize bias-variance trade-off by implementing the function `bias_variance_demo()` below:

In [13]:
from least_squares import least_squares
from split_data import split_data
from plots import bias_variance_decomposition_visualization

def bias_variance_demo():
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambd=1
    # define list to store the variable
    rmse_tr = np.empty((len(seeds), len(degrees)))
    rmse_te = np.empty((len(seeds), len(degrees)))
    
    for index_seed, seed in enumerate(seeds):  #sert à avoir plusieurs data différentes pour simuler plusieurs expériences
        np.random.seed(seed)
        x = np.linspace(0.1, 2 * np.pi, num_data)
        y = np.sin(x) + 0.3 * np.random.randn(num_data).T
        # ***************************************************
        # INSERT YOUR CODE HERE
        # split data with a specific seed: TODO
        St,Sv=split_data(x,y,ratio_train,seed)
        xt,yt=St
        xv,yv=Sv
        # ***************************************************
        # ***************************************************
        # INSERT YOUR CODE HERE
        # bias_variance_decomposition: TODO
        for index_def, degree in enumerate(degrees):
            
            xtpoly=build_poly(xt,degree)
            xvpoly=build_poly(xv,degree)
            wr_t=ridge_regression(yt,xtpoly,lambd)
            mse_t=compute_mse(yt,xtpoly,wr_t)
            mse_v=compute_mse(yv,xvpoly,wr_t)
    
            rmse_tr[index_seed,index_def]=np.power(mse_t*2,1/2)
            rmse_te[index_seed,index_def]=np.power(mse_v*2,1/2)
        # ***************************************************

    bias_variance_decomposition_visualization(degrees, rmse_tr, rmse_te)

bias_variance_demo()

In [23]:
xp=build_poly(x,5)
xp=reshape(xp,(len(xp),1))
print(np.transpose(xp[1,:]).shape)

NameError: name 'reshape' is not defined